In [1]:
q="jobname:*r7725.*"

In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from elasticsearch import Elasticsearch, exceptions as es_exceptions
from elasticsearch.helpers import scan
es = Elasticsearch(hosts=[{'host':'atlas-kibana.mwt2.org', 'port':9200}],timeout=60)

In [3]:
ind="jobs2016"

ss = {
  "size": 0,
  "aggs": {
    "2": {
      "terms": {
        "field": "jobstatus", "size": 5,
        "order": { "_count": "desc" }
      },
      "aggs": {
        "3": { "avg": { "field": "wall_time"  }   },
        "4": { "avg": { "field": "nevents"  } },
        "5": { "avg": {   "field": "cpuconsumptiontime"  }  },
        "6": { "avg": { "field": "corecount" }  },
        "7": { "avg": { "field": "inputfilebytes" }  },
        "8": { "sum": {  "field": "inputfilebytes" }  }
      }
    }
  },
  "query": {
    "filtered": {
      "query": {  "query_string": { "analyze_wildcard": True, "lowercase_expanded_terms": False,  "query": q }  }
    }
  }
}

In [4]:
res= es.search( index=ind, body=ss, analyze_wildcard=True)

# print(res)
for r in res['aggregations']['2']['buckets']:
    cores=r['6']['value']
    wall=r['3']['value']
    events=r['4']['value']
    cpu=r['5']['value']
    
    if events<1: continue
    secPerEvent=wall/events*cores
    cpueff=cpu/(wall*cores)
    print (r['key'], "\tjobs:", r['doc_count'],"\tevents:", events, '\t avg.cores:', cores)
    print ("wall time:", wall, "\twalltime per event per core:", secPerEvent, "[s/ev/core]" ) 
    print ("cpu time :", cpu,  "\tcputtime per event:", cpu/events ,"\tcpu eff.:", cpueff ) 
    print ('--------------------------------------------------------')


finished 	jobs: 503451 	events: 1997.8878341685686 	 avg.cores: 7.997650199524478
wall time: 20240.084095572358 	walltime per event per core: 81.02212238191582 [s/ev/core]
cpu time : 101666.42480996164 	cputtime per event: 50.886953246937736 	cpu eff.: 0.6280624569061614
--------------------------------------------------------
failed 	jobs: 165454 	events: 804.7216869945726 	 avg.cores: 7.99978846084108
wall time: 13881.255835458798 	walltime per event per core: 137.99442968813037 [s/ev/core]
cpu time : 33962.63133559781 	cputtime per event: 42.204195418716075 	cpu eff.: 0.305839848130814
--------------------------------------------------------
cancelled 	jobs: 2004 	events: 820.8582834331337 	 avg.cores: 8.0
wall time: 23957.48403193613 	walltime per event per core: 233.48716352583588 [s/ev/core]
cpu time : 80744.50449101797 	cputtime per event: 98.36594954407296 	cpu eff.: 0.4212906099790302
--------------------------------------------------------


In [5]:
ss = {
  "fields": ["wall_time","jobstatus"],
  "query": {
    "filtered": {
      "query": {  "query_string": { "analyze_wildcard": True, "lowercase_expanded_terms": False,  "query": q }  }
    }
  }
}

scroll = scan(es, query=ss, index=ind, scroll='10s')
fi=[]
fa=[]
for res in scroll:
    r=res['fields']
    if r['jobstatus'][0]=='finished':
        fi.append(r['wall_time'][0]/3600)
    elif r['jobstatus'][0]=='failed':
        fa.append(r['wall_time'][0]/3600)


ScanError: Scroll request has failed on 57 shards out of 460.

In [ ]:
plt.hist(fi,bins=30, log=True, alpha=0.5, label='finished')
plt.hist(fa,bins=30, log=True, alpha=0.5, label='failed')
plt.legend(loc='upper right')
plt.show()